In [1]:
# test.py
import sqlite3
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler

# Connect to database
DB_PATH = "../data/processed/chess_games.db"
conn = sqlite3.connect(DB_PATH)

# Get first game
query = """
SELECT 
    moves,
    white, black,
    white_elo, black_elo,
    eco, opening,
    result
FROM chess_games 
WHERE moves IS NOT NULL
  AND white_elo IS NOT NULL
  AND black_elo IS NOT NULL
LIMIT 1
"""


row = conn.execute(query).fetchone()

# Print game info
print("Game Information:")
print("-" * 40)
print(f"White: {row[1]} ({row[3]})")
print(f"Black: {row[2]} ({row[4]})")
print(f"ECO: {row[5]}")
print(f"Opening: {row[6]}")
print(f"Result: {row[7]}")
print("\nPGN:")
print(row[0])

# # change row 0 to cut moves till 8th move regex 8.
# moves = row[0].split(" ")[0:22]
# row = list(row)
# row[0] = " ".join(moves)
# row = tuple(row)
# print(row[0])

# Parse game
game = chess.pgn.read_game(io.StringIO(row[0]))

# Initialize feature extractor
feature_extractor = FeatureExtractor()

# Extract features without engine first
# print("\nFeatures without engine analysis:")
# print("-" * 40)
# features = feature_extractor.extract_features(game)
# for name, value in features.__dict__.items():
#     print(f"{name}: {value:.3f}")

# Initialize stockfish and get evaluations
stockfish = StockfishHandler(path="stockfish", depth=20)  # Update path
try:
    # Get positions and engine evaluations
    positions = feature_extractor._get_positions(game)
    print("\nPositions:")
    print(positions)
    print('-'*40)
    evals = [stockfish.evaluate_position(pos, i) for i, pos in enumerate(positions)]
    print("\nEngine evaluations:")
    print(evals)

    # Extract features with engine
    print("\nFeatures with engine analysis:")
    print("-" * 40)
    features_with_engine = feature_extractor.extract_features(game, evals)
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")
finally:
    stockfish.close()

conn.close()

Game Information:
----------------------------------------
White: Abdallah1990 (2510)
Black: EzWin78 (2647)
ECO: D00b
Opening: None
Result: 0-1

PGN:
1.d4 d5 2.c3 Nf6 3.c4 e6 4.Nf3 Bb4+ 5.Bd2 Be7 6.Nc3 O-O 7.cxd5 exd5 8.Bg5 b6 9.e3 Bb7 10.Bd3 Nbd7 11.O-O c5 12.Rc1 c4 13.Bb1 a6 14.Ne5 b5 15.a3 Nxe5 16.dxe5 Nd7 17.Bxe7 Qxe7 18.f4 Rad8 19.Nxd5 Qc5 20.Be4 Bxd5 21.Qxd5 Qxe3+ 22.Kh1 Nf6 23.exf6 Rxd5 24.fxg7 Rfd8 25.Bxd5 Rxd5 26.f5 Kxg7 27.f6+ Kg6 { Normal} 0-1

Positions:
[Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'), Board('rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1'), Board('rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2'), Board('rnbqkbnr/ppp1pppp/8/3p4/3P4/2P5/PP2PPPP/RNBQKBNR b KQkq - 0 2'), Board('rnbqkb1r/ppp1pppp/5n2/3p4/3P4/2P5/PP2PPPP/RNBQKBNR w KQkq - 1 3'), Board('rnbqkb1r/ppp1pppp/5n2/3p4/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 3'), Board('rnbqkb1r/ppp2ppp/4pn2/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 4'), Board('rnbqkb1r/ppp2ppp/4pn2/

In [1]:
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler
from analysis.move_analyzer import MoveAnalyzer
from models.data_classes import Info
from tqdm import tqdm

def print_move_analysis(ply: int, board: chess.Board, move: chess.Move, prev_info: Info, curr_info: Info, judgment: str):
    """Print detailed analysis of a move with best moves list"""
    move_number = (ply // 2) + 1
    is_white = ply % 2 == 0
    move_str = f"{move_number}." if is_white else f"{move_number}..."
    
    # Check if move is a sacrifice
    is_sacrifice = MoveAnalyzer.is_piece_sacrifice(board, move)
    sacrifice_str = " (SACRIFICE!)" if is_sacrifice else ""
    
    # Get player's move in SAN notation
    played_move_san = board.san(move)
    
    print(f"{move_str} {played_move_san}{sacrifice_str}")
    print(f"  Previous eval: {prev_info.eval}")
    print(f"  Current eval: {curr_info.eval}")
    print(f"  Judgment: {judgment}")
    
    # Print best moves if available
    if prev_info.variation:
        print("  Best moves:")
        for i, var in enumerate(prev_info.variation, 1):
            if 'Move' in var:
                move_uci = var['Move']
                try:
                    # Convert UCI move to SAN notation
                    board_copy = board.copy()
                    move_obj = chess.Move.from_uci(move_uci)
                    san_move = board_copy.san(move_obj)
                    
                    # Get evaluation for the move
                    eval_str = ""
                    if 'Centipawn' in var and var['Centipawn'] is not None:
                        cp = var['Centipawn']
                        eval_str = f" ({cp/100:+.2f})"
                    elif 'Mate' in var and var['Mate'] is not None:
                        mate = var['Mate']
                        eval_str = f" (#{mate})"
                    else:
                        eval_str = " (?)"  # Unknown evaluation
                    
                    # Highlight if this was the played move
                    played_indicator = " ←" if move_uci == move.uci() else ""
                    print(f"    {i}. {san_move}{eval_str}{played_indicator}")
                except (ValueError, AssertionError) as e:
                    print(f"    {i}. {move_uci} (Error: {str(e)})")
    
    # Print simplified evaluation line
    prev_eval = f"{prev_info.cp/100:+.1f}" if prev_info.cp is not None else "?"
    curr_eval = f"{curr_info.cp/100:+.1f}" if curr_info.cp is not None else "?"
    print(f"  {prev_eval} -> {curr_eval}")
    print("-" * 40)

# Moves input as a string (PGN format)
moves = """1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Be3 Bg7 5. Qd2 c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6
Bxh6 9. Qxh6 Bb7 10. a3 e5 11. O-O-O Qe7 12. Kb1 a6 13. Nc1 O-O-O 14. Nb3 exd4
15. Rxd4 c5 16. Rd1 Nb6 17. g3 Kb8 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1
d4 22. Nd5 Nbxd5 23. exd5 Qd6 24. Rxd4 cxd4 25. Re7+ Kb6 26. Qxd4+ Kxa5 27. b4+
Ka4 28. Qc3 Qxd5 29. Ra7 Bb7 30. Rxb7 Qc4 31. Qxf6 Kxa3 32. Qxa6+ Kxb4 33. c3+
Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 Rd2 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8
Rd3 40. Qa8 c3 41. Qa4+ Ke1 42. f4 f5 43. Kc1 Rd2 44. Qa7 1-0"""

# Create a file-like object from the moves string
pgn_io = io.StringIO(moves)

# Parse the game from the PGN string
game = chess.pgn.read_game(pgn_io)
if game is None:
    print("Failed to parse game!")
else:
    print("Game parsed successfully!\n")

# Print the PGN for verification
print("PGN:")
print(moves)
print("\n")

# Initialize the feature extractor and the Stockfish handler
feature_extractor = FeatureExtractor()
stockfish = StockfishHandler(path="stockfish", depth=20)

try:
    # Get positions from the game using the extractor method
    positions = feature_extractor._get_positions(game)
    
    # Get moves from the game
    moves_list = list(game.mainline_moves())
    
    # Evaluate each position using Stockfish with progress bar
    print("\nEvaluating positions...")
    evals = []
    for i, board in tqdm(enumerate(positions), total=len(positions), desc="Stockfish Analysis"):
        evals.append(stockfish.evaluate_position(board, i))
    
    print("\nDetailed Move Analysis:")
    print("=" * 40)
    
    # Analyze each move
    for i in range(1, len(evals)):
        if i-1 >= len(moves_list):
            break
            
        prev, curr = evals[i-1], evals[i]
        board = positions[i-1]
        move = moves_list[i-1]
        
        judgment = MoveAnalyzer.analyze_move(prev, curr, board, move)
        print_move_analysis(i-1, board, move, prev, curr, judgment)
    
    # Extract features using both the game and engine evaluations
    features_with_engine = feature_extractor.extract_features(game, evals)
    print("\nFeatures with engine analysis:")
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")

finally:
    stockfish.close()

Game parsed successfully!

PGN:
1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Be3 Bg7 5. Qd2 c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6
Bxh6 9. Qxh6 Bb7 10. a3 e5 11. O-O-O Qe7 12. Kb1 a6 13. Nc1 O-O-O 14. Nb3 exd4
15. Rxd4 c5 16. Rd1 Nb6 17. g3 Kb8 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1
d4 22. Nd5 Nbxd5 23. exd5 Qd6 24. Rxd4 cxd4 25. Re7+ Kb6 26. Qxd4+ Kxa5 27. b4+
Ka4 28. Qc3 Qxd5 29. Ra7 Bb7 30. Rxb7 Qc4 31. Qxf6 Kxa3 32. Qxa6+ Kxb4 33. c3+
Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 Rd2 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8
Rd3 40. Qa8 c3 41. Qa4+ Ke1 42. f4 f5 43. Kc1 Rd2 44. Qa7 1-0



Evaluating positions...


Stockfish Analysis: 100%|██████████| 88/88 [06:49<00:00,  4.65s/it]


Detailed Move Analysis:
1. e4
  Previous eval: {'type': 'cp', 'value': 19}
  Current eval: {'type': 'cp', 'value': 33}
  Judgment: Judgment.GOOD
  Best moves:
    1. e4 (+0.34) ←
    2. d4 (+0.22)
    3. Nf3 (+0.22)
  +0.2 -> +0.3
----------------------------------------
1... d6
  Previous eval: {'type': 'cp', 'value': 33}
  Current eval: {'type': 'cp', 'value': 38}
  Judgment: Judgment.GOOD
  Best moves:
    1. e5 (+0.24)
    2. c5 (+0.32)
    3. e6 (+0.44)
  +0.3 -> +0.4
----------------------------------------
2. d4
  Previous eval: {'type': 'cp', 'value': 38}
  Current eval: {'type': 'cp', 'value': 76}
  Judgment: Judgment.GOOD
  Best moves:
    1. d4 (+0.48) ←
    2. Ne2 (+0.37)
    3. Nf3 (+0.35)
  +0.4 -> +0.8
----------------------------------------
2... Nf6
  Previous eval: {'type': 'cp', 'value': 76}
  Current eval: {'type': 'cp', 'value': 31}
  Judgment: Judgment.GOOD
  Best moves:
    1. e5 (+0.60)
    2. Nf6 (+0.62) ←
    3. g6 (+0.70)
  +0.8 -> +0.3
---------------------